In [1]:
# Python tool to perform calculations of HPL parameters
# following instructions by Mohamad Sindi, documented at https://sourceforge.net/projects/hpl-calculator/
# basically a copy of http://hpl-calculator.sourceforge.net/ but in not PHP

In [27]:
params = {
    # number of nodes to be benchmarked
    'NODE_COUNT': 12,
    # number of cpu cores in every node
    'CORES_PER_NODE': 32,
    # speed of every cpu core, in GHz (should be same for all cores)
    'SPEED_PER_CORE': 1.9,
    # memory available for every cpu core, in GB
    'MEM_PER_NODE': 93,
    # instructions per cycle for cpu
    'INSTR_PER_CYCLE': 32
}

In [13]:
import pandas as pd
import numpy as np

In [14]:
def PQ_values(par):
    """
    Calculates all possible pairs of P and Q, the grid sizes
    P*Q should equal the number of total processors in the grid,
    and P and Q should be as close together as possible
    """
    # solving NP problems with naive algorithms
    cores = par['NODE_COUNT'] * par['CORES_PER_NODE']
    ps = []
    qs = []
    for i in range(1, int(np.sqrt(cores)) + 1):
        if cores % i == 0:
            ps.append(i)
            qs.append(int(cores / i))
    return pd.DataFrame({'P': ps, 'Q': qs})

In [15]:
PQ_values(params)

,P,Q
0,1,384
1,2,192
2,3,128
3,4,96
4,6,64
5,8,48
6,12,32
7,16,24


In [16]:
def N_values(par):
    """
    Returns a table containing the values of N and NB, the memory
    usage percentage and the grid size.
    As with the original web tool, our values of N range from 80% to 100%,
    and NB ranges from 96 to 256.
    """
    cols = {}
    npcts = range(80, 100 + 1, 2)
    for nb in range(96, 256 + 1, 8):
        cols[nb] = [
            int(npct / 100 * np.sqrt(par['MEM_PER_NODE'] * 1024 * 1024 * 1024 * par['NODE_COUNT'] / 8) / nb)
            * nb # aligns NB or something
            for npct in npcts
        ]
    return pd.DataFrame(data=cols, index=npcts)

In [17]:
N_values(params)

,96,104,112,120,128,136,144,152,160,168,...,184,192,200,208,216,224,232,240,248,256
80,309600,309608,309568,309600,309504,309536,309600,309472,309600,309456,...,309488,309504,309600,309504,309528,309568,309488,309600,309504,309504
82,317280,317304,317296,317280,317312,317288,317232,317224,317280,317352,...,317216,317184,317200,317200,317304,317184,317144,317280,317192,317184
84,325056,325000,325024,325080,324992,325040,325008,324976,324960,325080,...,324944,325056,325000,324896,325080,325024,325032,324960,324880,324864
86,332832,332800,332752,332760,332800,332792,332784,332728,332800,332808,...,332672,332736,332800,332800,332640,332640,332688,332640,332816,332800
88,340512,340496,340480,340560,340480,340544,340560,340480,340480,340536,...,340400,340416,340400,340496,340416,340480,340576,340560,340504,340480
90,348288,348296,348320,348240,348288,348296,348192,348232,348320,348264,...,348312,348288,348200,348192,348192,348320,348232,348240,348192,348160
92,355968,355992,356048,356040,355968,356048,355968,355984,356000,355992,...,356040,355968,356000,355888,355968,355936,355888,355920,355880,355840
94,363744,363792,363776,363720,363776,363800,363744,363736,363680,363720,...,363768,363648,363800,363792,363744,363776,363776,363600,363568,363776
96,371520,371488,371504,371520,371456,371416,371520,371488,371520,371448,...,371496,371520,371400,371488,371520,371392,371432,371520,371504,371456
98,379200,379184,379232,379200,379264,379168,379152,379240,379200,379176,...,379224,379200,379200,379184,379080,379232,379088,379200,379192,379136


In [28]:
def Rpeak(par):
    """
    Calculates the theoretical peak performance of the system in GFLOPS.
    """
    return par['SPEED_PER_CORE'] * par['NODE_COUNT'] * par['CORES_PER_NODE'] * par['INSTR_PER_CYCLE']

In [29]:
Rpeak(params)

23347.199999999997

In [30]:
def efficiencies(par):
    """
    Calculates possible performance levels in GFLOPS.
    """
    peak = Rpeak(par)
    pcts = range(70, 100 + 1, 2)
    return pd.DataFrame(data={'GHz': [pct / 100 * peak for pct in pcts]}, index=pcts)

In [31]:
efficiencies(params)

,GHz
70,16343.040
72,16809.984
74,17276.928
76,17743.872
78,18210.816
80,18677.760
82,19144.704
84,19611.648
86,20078.592
88,20545.536
